In [1]:
import pandas as pd
import glob, os 

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
import warnings
warnings.simplefilter('ignore')

In [6]:
path = 'F:\CS_599\social-swear-master\social-swear-master\datasets\\tweets-5M_withDate_wave2.csv'

In [8]:
tweets_raw = pd.DataFrame()
sets = [[250,1000],[1001,5000]]
w_set = 1
df = pd.read_csv(path,index_col=None, header=0)
df = df.loc[(df['user_followers_count'] >= sets[w_set][0]) & (df['user_followers_count'] <= sets[w_set][1])]
tweets_raw = pd.concat([tweets_raw,df])
del df

In [9]:
len(tweets_raw)

1269900

In [10]:
'''
allFiles = glob.glob(path + "/*.csv")
tweets_raw = pd.DataFrame()
sets = [[250,1000],[1001,5000]]
w_set = 1      #change to use different set of data
for file_ in allFiles:
    df = pd.read_csv(file_,index_col=None, header=0)
    df = df.loc[(df['user_followers_count'] >= sets[w_set][0]) & (df['user_followers_count'] <= sets[w_set][1])]
    tweets_raw = pd.concat([tweets_raw,df])
    del df
    '''

'\nallFiles = glob.glob(path + "/*.csv")\ntweets_raw = pd.DataFrame()\nsets = [[250,1000],[1001,5000]]\nw_set = 1      #change to use different set of data\nfor file_ in allFiles:\n    df = pd.read_csv(file_,index_col=None, header=0)\n    df = df.loc[(df[\'user_followers_count\'] >= sets[w_set][0]) & (df[\'user_followers_count\'] <= sets[w_set][1])]\n    tweets_raw = pd.concat([tweets_raw,df])\n    del df\n    '

Column descriptions:
- Tweet content:
    -    `text`: raw text of the tweet, unchanged
    -    `text_noMention`': text of the tweet with @mentions removed (='NO_USER_MENTIONS' if 0 mentions)
    -    `numMentions`: number of mentions found in the tweet



- Tweet metadata:
    -    `is_quote_status`: whether the tweet is a quote status of another tweet
    -    `is_reply_to_status`: whether the tweet is a reply to someone elses tweet
    -    `is_reply_to_user`: whether the tweet is a reply to a user


- Account information:
    -    `user_followers_count`: number of followers the user has
    -    `user_friends_count`: number of people the user follows
    -    `user_listed_count`: number of lists the user is a part of
    -    `user_favourites_count`: number of tweets the user has liked to date
    -    `user_statuses_count`: number of tweets this user has authored to date (note that timeline acquisition is limited to 3200 latest tweets)
    -    `user_verified`: whether the user is a verified user
    -    `user_description_text`: raw text of the user's profile description
    


- Engagement:
    -    `retweet_count`: number of times this tweet has been retweeted (at time of collection)
    -    `favorite_coun`': number of times this tweet has been liked (at time of collection)

In the future:
- [ ] date
- [ ] location (from geo-coordinates or timezone)

## Curate dataframe

In [11]:
#Create new column objective with weighted sum of favorite and retweet
tweets_raw['retweet_count']*= 5
tweets_raw['engagment'] = tweets_raw['favorite_count'] + tweets_raw['retweet_count']

In [19]:
tweets = tweets_raw[[
    'user_id',
    'text',
    'retweet_count',
    'favorite_count',
    'user_followers_count',
    'user_friends_count',
    'user_statuses_count',
    'numMentions',
    'date',
    'engagment'
]].rename({
    'retweet_count': 'n_retweets',
    'favorite_count': 'n_favorites',
    'user_followers_count': 'n_user_followers',
    'user_friends_count': 'n_user_following',
    'user_statuses_count': 'n_user_posts'
}, axis=1)

In [20]:
tweets['is_reply'] = tweets_raw.is_reply_to_user.astype(bool) | tweets_raw.is_reply_to_status.astype(bool)

In [21]:
int_cols = ['n_retweets', 'n_favorites', 'n_user_followers', 'n_user_following', 'n_user_posts']
for col in int_cols:
    tweets[col] = tweets[col].astype(int, errors='ignore')
tweets.dropna(inplace=True)

In [22]:
import numpy as np

In [23]:
# no replies (under 1% percent) likely means that the user is a bot
per_replies = tweets.groupby('user_id').is_reply.mean()
bots_ids = per_replies[per_replies < .01].index
tweets['is_user_bot'] = tweets.user_id.isin(bots_ids)
tweets['is_outlier'] = False
tweets['is_valid'] = ~tweets.is_user_bot & ~tweets.is_outlier

## Find outliers

In [24]:
tweets['is_valid'] = ~tweets.is_user_bot #& ~tweets.is_outlier did not use in users over 5001 n_user_followers

In [25]:
valid_tweets = tweets[tweets.is_valid]
del tweets

# Pre-process

In [26]:
#Remove Duplicates
valid_tweets = valid_tweets[~valid_tweets.index.duplicated()]

In [27]:
#Create Dict of all tweets
tweet_dictionary = {}
i = 0
for line in valid_tweets['text']:
        tweet_dictionary[i] = line.lower()
        i += 1

In [28]:
%%time
#strip links
import re
def strip_links(text):
    link_regex    = re.compile('((https?):((//)|(\\\\))+([\w\d:#@%/;$()~_?\+-=\\\.&](#!)?)*)', re.DOTALL)
    links         = re.findall(link_regex, text)
    for link in links:
        text = text.replace(link[0], ', ')    
    return text

for i in range(0,len(tweet_dictionary)):
    tweet_dictionary[i]=strip_links(tweet_dictionary[i])

CPU times: user 3.5 s, sys: 15 ms, total: 3.51 s
Wall time: 3.52 s


In [29]:
cList = {
  "ain't": "am not",
  "aren't": "are not",
  "can't": "cannot",
  "can't've": "cannot have",
  "'cause": "because",
  "could've": "could have",
  "couldn't": "could not",
  "couldn't've": "could not have",
  "didn't": "did not",
  "doesn't": "does not",
  "don't": "do not",
  "hadn't": "had not",
  "hadn't've": "had not have",
  "hasn't": "has not",
  "haven't": "have not",
  "he'd": "he would",
  "he'd've": "he would have",
  "he'll": "he will",
  "he'll've": "he will have",
  "he's": "he is",
  "how'd": "how did",
  "how'd'y": "how do you",
  "how'll": "how will",
  "how's": "how is",
  "I'd": "I would",
  "I'd've": "I would have",
  "I'll": "I will",
  "I'll've": "I will have",
  "i'm": "i am",
  "I've": "I have",
  "isn't": "is not",
  "it'd": "it had",
  "it'd've": "it would have",
  "it'll": "it will",
  "it'll've": "it will have",
  "it's": "it is",
  "let's": "let us",
  "ma'am": "madam",
  "mayn't": "may not",
  "might've": "might have",
  "mightn't": "might not",
  "mightn't've": "might not have",
  "must've": "must have",
  "mustn't": "must not",
  "mustn't've": "must not have",
  "needn't": "need not",
  "needn't've": "need not have",
  "o'clock": "of the clock",
  "oughtn't": "ought not",
  "oughtn't've": "ought not have",
  "shan't": "shall not",
  "sha'n't": "shall not",
  "shan't've": "shall not have",
  "she'd": "she would",
  "she'd've": "she would have",
  "she'll": "she will",
  "she'll've": "she will have",
  "she's": "she is",
  "should've": "should have",
  "shouldn't": "should not",
  "shouldn't've": "should not have",
  "so've": "so have",
  "so's": "so is",
  "that'd": "that would",
  "that'd've": "that would have",
  "that's": "that is",
  "there'd": "there had",
  "there'd've": "there would have",
  "there's": "there is",
  "they'd": "they would",
  "they'd've": "they would have",
  "they'll": "they will",
  "they'll've": "they will have",
  "they're": "they are",
  "they've": "they have",
  "to've": "to have",
  "wasn't": "was not",
  "we'd": "we had",
  "we'd've": "we would have",
  "we'll": "we will",
  "we'll've": "we will have",
  "we're": "we are",
  "we've": "we have",
  "weren't": "were not",
  "what'll": "what will",
  "what'll've": "what will have",
  "what're": "what are",
  "what's": "what is",
  "what've": "what have",
  "when's": "when is",
  "when've": "when have",
  "where'd": "where did",
  "where's": "where is",
  "where've": "where have",
  "who'll": "who will",
  "who'll've": "who will have",
  "who's": "who is",
  "who've": "who have",
  "why's": "why is",
  "why've": "why have",
  "will've": "will have",
  "won't": "will not",
  "won't've": "will not have",
  "would've": "would have",
  "wouldn't": "would not",
  "wouldn't've": "would not have",
  "y'all": "you all",
  "y'alls": "you alls",
  "y'all'd": "you all would",
  "y'all'd've": "you all would have",
  "y'all're": "you all are",
  "y'all've": "you all have",
  "you'd": "you had",
  "you'd've": "you would have",
  "you'll": "you will",
  "you'll've": "you will have",
  "you're": "you are",
  "you've": "you have"
}

c_re = re.compile('(%s)' % '|'.join(cList.keys()))

def expandContractions(text, c_re=c_re):
    def replace(match):
        return cList[match.group(0)]
    text = c_re.sub(replace, text)
    return text

In [30]:
%%time
for i in range(0,len(tweet_dictionary)):
    tweet_dictionary[i]=expandContractions(tweet_dictionary[i])

CPU times: user 13.4 s, sys: 35.1 ms, total: 13.4 s
Wall time: 13.4 s


In [32]:
%%time
#Remove Mentions
import string
def strip_mentions(text):
    entity_prefixes = ['@']
    words = []
    for word in text.split():
        word = word.strip()
        if word:
            if word[0] not in entity_prefixes:
                words.append(word)
    return ' '.join(words)

for i in range(0,len(tweet_dictionary)):
    tweet_dictionary[i]=strip_mentions(tweet_dictionary[i])

CPU times: user 3.31 s, sys: 7.29 ms, total: 3.32 s
Wall time: 3.32 s


In [34]:
%%time
#Remove Hashtags
def strip_hashtags(text):
    entity_prefixes = ['#']
    words = []
    for word in text.split():
        word = word.strip()
        if word:
            if word[0] not in entity_prefixes:
                words.append(word)
    return ' '.join(words)

for i in range(0,len(tweet_dictionary)):
    tweet_dictionary[i]=strip_hashtags(tweet_dictionary[i])

CPU times: user 3.16 s, sys: 6.59 ms, total: 3.16 s
Wall time: 3.16 s


In [36]:
#preprocess for censored swear detection
censoredSwearDict = tweet_dictionary.copy()
for i in range(0,len(tweet_dictionary)):
    censoredSwearDict[i]= censoredSwearDict[i].replace(r"[,.”“]",'')

In [18]:
import emoji
from emoji import UNICODE_EMOJI

In [19]:
%%time
#demoji text
import emoji
for i in range(0,len(tweet_dictionary)):
    tweet_dictionary[i]=emoji.demojize(tweet_dictionary[i])

Wall time: 6min 23s


In [20]:
se = pd.Series(tweet_dictionary)
valid_tweets['demoji_text'] = se.values

In [21]:
import string
def eggplant_presence(text):
    entity_prefixes = ':eggplant:'
    for word in text.split():
        if word == entity_prefixes:
            return 1
    return 0

def mfinger_presence(text):
    entity_prefixes = ':middle_finger:'
    for word in text.split():
        if word == entity_prefixes:
            return 1
    return 0
def cursing_presence(text):
    entity_prefixes = ':cursing:'
    for word in text.split():
        if word == entity_prefixes:
            return 1
    return 0
def fuck_presence(text):
    entity_prefixes = ':point_right::ok_hand:'
    for word in text.split():
        if word == entity_prefixes:
            return 1
    return 0

In [22]:
%%time 
valid_tweets['eggplant'] = valid_tweets.demoji_text.apply(lambda t: eggplant_presence(t))
valid_tweets['mfinger'] = valid_tweets.demoji_text.apply(lambda t: mfinger_presence(t))
valid_tweets['curse'] = valid_tweets.demoji_text.apply(lambda t: cursing_presence(t))
valid_tweets['fuck'] = valid_tweets.demoji_text.apply(lambda t: fuck_presence(t))
del valid_tweets['demoji_text']

Wall time: 11.7 s


In [28]:
%%time
#Remove Special characters
import unicodedata
def remove_special_characters(text, remove_digits=False):
    pattern = r'[^a-zA-z0-9\s]' if not remove_digits else r'[^a-zA-z\s]'
    text = re.sub(pattern, '', text)
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text

for i in range(0,len(tweet_dictionary)):
    tweet_dictionary[i]=remove_special_characters(tweet_dictionary[i], remove_digits=True)

Wall time: 5.38 s


In [29]:
from textblob import TextBlob

In [30]:
#Text Blob sentiment and obj columns

polarity=[]
subjectivity=[]

for i in range(0,len(tweet_dictionary)):
    snt = TextBlob(tweet_dictionary[i])
    polarity.append(snt.sentiment.polarity)
    subjectivity.append(snt.sentiment.subjectivity)

In [31]:
se = pd.Series(polarity)
valid_tweets['text_polarity'] = se.values
se = pd.Series(subjectivity)
valid_tweets['text_subjectivity'] = se.values

In [ ]:

#Create Vader sentiment columns

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

comp=[]
pos=[]
neu=[]
neg=[]
for i in range(0,len(valid_tweets)):
    snt = analyser.polarity_scores(tweet_dictionary[i])
    #print(i)
    #print(tweetData.at[i,'vader_comp'])
    comp.append(snt['compound'])
    pos.append(snt['pos'])
    neu.append(snt['neu'])
    neg.append(snt['neg'])

In [ ]:
se = pd.Series(comp)
valid_tweets['vader_comp'] = se.values
se = pd.Series(pos)
valid_tweets['vader_pos'] = se.values
se = pd.Series(neu)
valid_tweets['vader_neu'] = se.values
se = pd.Series(neg)
valid_tweets['vader_neg'] = se.values

In [ ]:
#processed tweets to df
se = pd.Series(tweet_dictionary)
valid_tweets['processed_text'] = se.values

In [ ]:
from stop_words import get_stop_words
from nltk.corpus import stopwords

stop_words = list(get_stop_words('en'))         #About 900 stopwords
nltk_words = list(stopwords.words('english')) #About 150 stopwords
stop_words.extend(nltk_words)

def remove_stop_words(word_list):
    output = [w for w in word_list if not w in stop_words]
    return output

In [ ]:
# tokenize, lemmatize, lowercase
%time valid_tweets['text_token'] = valid_tweets.processed_text.apply(lambda t: TextBlob(t).words.lemmatize())

In [ ]:
%time valid_tweets['text_token_rem_stop'] = valid_tweets.text_token.apply(lambda t: remove_stop_words(t))

In [ ]:
valid_tweets['text_len'] = valid_tweets.text.str.len()

In [ ]:
# rudimentary: catches nearly all cases, but can be further refined
valid_tweets['n_mentions'] = valid_tweets.text.str.count('@')
valid_tweets['n_hashtags'] = valid_tweets.text.str.count('#')
valid_tweets['n_links']    = valid_tweets.text.str.count('http')
valid_tweets['n_emojis'] = valid_tweets.text.apply(emoji.emoji_count)

In [ ]:
import csv 
with open('swearWordsSeverityDict.txt', mode='r') as infile:
    reader = csv.reader(infile)
    severityDict = {columns[0]:columns[1] for columns in reader}

In [37]:
swearDict = dict()

# function to count swears per tweet
def countSwears(inputSentence):
    filteredSentence = []
    if inputSentence != inputSentence:
        return filteredSentence
    wordTokens = inputSentence.split(' ')
    for w in wordTokens:
        if w in swearWords:
            filteredSentence.append(w)
            if w in swearDict: 
                swearDict[w] = swearDict[w]+1
            else:
                swearDict[w] = 1
    return filteredSentence

In [ ]:
# function to count swears per tweet
def countSwearsWithoutIncrement(inputSentence):
    filteredSentence = []
    if inputSentence != inputSentence:
        return filteredSentence
    wordTokens = inputSentence.split(' ')
    for w in wordTokens:
        if w in swearWords:
            filteredSentence.append(w)
    return filteredSentence

In [39]:
# Store swear words
f = open('swearWords.txt', 'r+')
swearWords = [line.strip() for line in f.readlines()]
f.close()

outputDF = pd.DataFrame()
swearCount = 0
swear_tweets = 0
nonswear_tweets = 0
presenceList = []
countList = []
severityList = []
rarityList = []
# preprocess tweets (makes a copy of the 'text' field, does not preprocess tweetData['text'])
for i in range(0,len(valid_tweets)):
    swearList = countSwears(tweet_dictionary[i])
     count = len(swearList)
    swearCount += count
    severity = 0
    for word in swearList:
        severity += int(severityDict.get(word))
    if count != 0:
        severity = severity/count
    else: 
        severity = 0
    severityList.append(severity)
    countList.append(count)
    if count == 0:
        presenceList.append(0)
        nonswear_tweets += 1
    else:
        presenceList.append(1)
        swear_tweets += 1
        
for i in range(0,len(df_list[0])):
    swearList = countSwearsWithoutIncrement(tweet_dictionary[i])
    count = len(swearList)
    rarity = 0
    for word in swearList:
        rarity += swearDict.get(word)/swearCount
    if count != 0:
        rarity = rarity/count
    else:
        rarity = 0
    rarityList.append(rarity)

countList = pd.Series(countList)
presenceList = pd.Series(presenceList)
severityList = pd.Series(severityList)
rarityList = pd.Series(rarityList)



valid_tweets['swear_count'] = countList.values
valid_tweets['swear_present'] = presenceList.values
valid_tweets['swear_severity'] = severityList.values
valid_tweets['swear_rarity_by_percentage'] = rarityList.values

In [40]:
# Store files of censored swear words in adjusted form
# truncated - all censored characters truncated/compressed
# original length - substitution of each censored character
# special chars - censored swears that contain runs of common chars #$@&%!*

with open('censoredSwearsTruncated.txt', 'r') as f:
    censorTruncatedList = f.read().splitlines()
with open('censoredSwearsOriginalLength.txt', 'r') as f:
    censorOriginalList = f.read().splitlines()
with open('censoredSwearsSpecialChars.txt', 'r') as f:
    censoredSpecialCharsList = f.read().splitlines()

In [41]:
# converts each string into a form to be compared with the lists above
# for original length - converts all special chars to * (example from f$%&*r --> f****r)
# for truncated length - converts all runs of special chars to * (example from f$%&*r --> f*r)
# for censored chars - converts all special censor chars to ~, but a run of special chars only counts as 1
# (example f##$()r --> f**r). The reason for this case is to prevent detection of random runs of characters 
# as being detected as swears, but also the usage of the specific special chars still need to be detected. 
# example: how is !!!! (not swear) different from !@#$ (swear). The adjusted for of !!!! is ~ (since no changes)
# whereas the adjusted form of !@#$ is ~~~~ which is (which matches an entry from the list). The method is not perfect
# but works in most cases.
censoredSwearList = []
def censoredSwearDetector(inputList):
    newList = []
    for w in inputList:
        if w != '18-hole' and w != '36-hole'and  w != '54-hole' and w != '72-hole' and w != '#c25k' and w not in censoredSpecialCharsList:
            transformedWordOriginalLength = re.sub('[^a-z]', '*', w)
            transformedWordTruncatedLength = re.sub('[^a-z]+', '*', w)
            transformedCensoredChars = re.sub('[#]+', '~', w)
            transformedCensoredChars = re.sub('[$]+', '~', transformedCensoredChars)
            transformedCensoredChars = re.sub('[@]+', '~', transformedCensoredChars)
            transformedCensoredChars = re.sub('[&]+', '~', transformedCensoredChars)
            transformedCensoredChars = re.sub('[%]+', '~', transformedCensoredChars)
            transformedCensoredChars = re.sub('[!]+', '~', transformedCensoredChars)
            transformedCensoredChars = re.sub('[*]+', '~', transformedCensoredChars)
            if transformedWordTruncatedLength in censorTruncatedList:
                #print(w + '   truncated length   ' + transformedWordTruncatedLength)
                censoredSwearList.append(w)
                return 1
            elif transformedWordOriginalLength in censorOriginalList:
                #print(w + '   original length')
                censoredSwearList.append(w)
                return 1
            elif w == '@ss' or w == '*@ss' or w == '@ss*':
                #print(w)
                censoredSwearList.append(w)
                return 1
            elif transformedCensoredChars in censoredSpecialCharsList:
                #print(w + '   censored characters')
                censoredSwearList.append(w)
                return 1
    return 0

In [42]:
# filtering sentences for censored swear word detection. Must not be purely alphabetic or purely numeric
def filterWordsCensored(inputSentence):
    wordTokens = inputSentence.split(' ')
    filteredSentence = []
    for w in wordTokens:
        if not w.isalpha() and not w.isspace() and not w.isnumeric():
            filteredSentence.append(w)
    return filteredSentence

In [43]:
# apply fitlering
for i in range(0,len(valid_tweets)):
    censoredSwearDict[i] = filterWordsCensored(censoredSwearDict[i])

In [45]:
# create column of censored swear presence
presenceCensoredList = []
for i in range(0,len(valid_tweets)):
    count = censoredSwearDetector(censoredSwearDict[i])
    presenceCensoredList.append(count)

presenceCensoredList = pd.Series(presenceCensoredList)
valid_tweets['censored_presence'] = presenceCensoredList.values

In [1]:
print(censoredSwearList)

NameError: name 'censoredSwearList' is not defined

In [ ]:

from dateutil.parser import parse as parse_date

# parse dates
%time valid_tweets.date = valid_tweets.date.apply(parse_date)

%time valid_tweets['day_of_week'] = valid_tweets.date.apply(lambda d: int(d.strftime('%w')))

%time valid_tweets['hour_of_day'] = valid_tweets.date.apply(lambda d: int(d.strftime('%H')))


In [ ]:
%time valid_tweets.to_csv('50M_250_1000_2.csv', index=False)